In [5]:
import cv2
from cv2 import destroyAllWindows
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [6]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [7]:
def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [12]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time

# Initialize Mediapipe Pose and Drawing modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    vec1 = np.array(a) - np.array(b)
    vec2 = np.array(c) - np.array(b)
    radians = np.arctan2(np.linalg.norm(np.cross(vec1, vec2)), np.dot(vec1, vec2))
    angle = np.degrees(radians)
    return angle

# def find_camera_index():
#     for i in range(20):  # You can adjust the range based on your system's available devices
#         cap = cv2.VideoCapture(i)
#         if cap.isOpened():
#             print(f"Camera index {i} is available.")
#             cap.release()
#         else:
#             print(f"Camera index {i} is not available.")

# find_camera_index()
# def find_available_cameras():
#     available_cameras = []
#     for i in range(10):  # Try indexing from 0 to 9 (adjust as needed)
#         cap = cv2.VideoCapture(i)
#         if cap.isOpened():
#             available_cameras.append(i)
#             cap.release()
#     return available_cameras
            
# cap1 = cv2.VideoCapture(1)  # Use appropriate camera index here

def find_available_cameras():
    available_cameras = []
    for i in range(10):  # Try indexing from 0 to 9 (adjust as needed)
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()
    return available_cameras

# Example: Find available cameras and select the first two
available_cameras = find_available_cameras()
if len(available_cameras) >= 1:
    cap1 = cv2.VideoCapture(available_cameras[0])
    print(f"Capturing from camera {available_cameras[0]}")
else:
    print("Not enough cameras found.")
    

counter1 = 0
stage1 = None

angle_values1 = []

size1 = (640, 480)

frame_rate1 = cap1.get(cv2.CAP_PROP_FPS)

cv2.waitKey(200)
cap1.set(cv2.CAP_PROP_FPS, 2)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out1 = cv2.VideoWriter('camera1_output.mp4', fourcc, 5, size1)

duration = 10  # 10 seconds

start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap1.isOpened():
        elapsed_time = time.time() - start_time
        current_time = time.time() - start_time
        
#         if elapsed_time >= duration:
#             print("Time period elapsed. Stopping the loop.")
#             break
            
        ret1, frame1 = cap1.read()
        
        if not ret1:
            break
        
        image1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        
        image1.flags.writeable = False
        results1 = pose.process(image1)
        image1.flags.writeable = True
        
        image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
        
        try:
            if results1.pose_landmarks is not None:
                landmarks1 = results1.pose_landmarks.landmark

                shoulder1 = [landmarks1[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow1 = [landmarks1[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist1 = [landmarks1[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                hip1 = [landmarks1[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee1 = [landmarks1[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle1 = [landmarks1[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                angle_knee1 = calculate_angle(hip1, knee1, ankle1)
                angle_hip1 = calculate_angle(shoulder1, hip1, knee1)
                hip_angle1 = 180 - angle_hip1
                knee_angle1 = 180 - angle_knee1

                # Visualization and counter logic
                cv2.putText(image1, str(angle_knee1),
                            tuple(np.multiply(knee1, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (79, 121, 66), 2, cv2.LINE_AA)

                if angle_knee1 > 169:
                    stage1 = "UP"
                if angle_knee1 <= 90 and stage1 == 'UP':
                    stage1 = "DOWN"
                    counter1 += 1
                    print(counter1)

                # Drawing landmarks and connections
                mp_drawing.draw_landmarks(image1, results1.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                          mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

                out1.write(image1)
                angle_values1.append({'Time': current_time, 'Camera 1 Angle': angle_knee1})
                
        except Exception as e:
            print("Error:", e)
            
        cv2.imshow('Camera 1', image1)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap1.release()
cv2.destroyAllWindows()

# Create DataFrames from the angle lists
df1 = pd.DataFrame(angle_values1)

try:
    df1.to_excel('camera1_angles.xlsx', index=False)
    print("Excel files saved successfully.")
except Exception as e:
    print("Error saving Excel files:", e)

try:
    out1.release()
    print("Video files saved successfully.")
except Exception as e:
    print("Error saving Video files:", e)


Capturing from camera 0
Excel files saved successfully.
Video files saved successfully.
